In [1]:
import numpy as np
import pickle
import pylada.crystal.read as pcread
import pylada.crystal.write as pcwrite
from copy import deepcopy
from util import write_tcl
import numpy.linalg as npl
from pylada.crystal import supercell, space_group, into_cell
from util import rot_euler
from copy import deepcopy



class Options():
    def __init__(self):
        self.output_tiles = 1
options = Options()


def basic_tform_cell(M, A):
    # transform cell by a left multiplication
    from copy import deepcopy
    B = deepcopy(A)
    B.cell = np.dot(M, A.cell)
    for b in B:
        b.pos = into_cell(np.dot(M, b.pos), B.cell)
    return B



def eq_struct(s1,s2):
    eqc = npl.norm(s1.cell-s2.cell) < 1e-6
#    vals = [npl.norm(s1[k].pos - s2[k].pos) for k in range(len(s1))]
#    print vals
    vals = [npl.norm(s1[k].pos - s2[k].pos) < 1e-6 for k in range(len(s1))]
#    print vals
    eqp = all(vals)
    return eqc and eqp
    



In [15]:

def load_sym_dict(symloc):
    with open(symloc,'rb') as handle:
        syms=pickle.loads(handle.read())
    return syms


def load_syms(symloc, space_group):
    with open(symloc,'rb') as handle:
        syms=pickle.loads(handle.read())[space_group]
    return syms

def load_all_syms():
    ## this is the space group that includes all 192 possible operations
    symidx = 227
#    symidx = 168
    syms = load_syms("symmetries.pickle", symidx)
    return syms

syms = load_all_syms()
symdict = load_sym_dict("symmetries.pickle")

print len(syms)

s = pcread.poscar("symtest.vasp")
print s

192
Structure( 10, 5, 0,
           5, 10, 0,
           0, 0, 10,
           scale=1, name='icsd_100672.cif' )\
  .add_atom(8.5, 5, 0, 'Al')\
  .add_atom(4.5, 4.5, 0, 'Sn')


In [2]:

s2 = basic_tform_cell(syms[symidx], s)
print s2
pcwrite.poscar(s2, "sym2.vasp", vasp5=True)

write_tcl(options,s,s2,[],"nb1")
M = np.dot(s.cell, npl.inv(s2.cell))
s3 = basic_tform_cell(M, s2)
print s3
write_tcl(options,s,s3,[],"nb2")

NameError: name 'syms' is not defined

In [5]:
eq = []
for i in range(len(syms)):
    s2 = basic_tform_cell(syms[i], s)
    M = np.dot(s.cell, npl.inv(s2.cell))
    s3 = basic_tform_cell(M, s2)
    if (not eq_struct(s,s3)):
        print s
        print "AND"
        print s3
        print "are not equal"
    else:
        eq.append(i)
print len(eq), "ARE equal"

192 ARE equal


In [ ]:

s2 = pcread.poscar("symtest.sym.vasp")
s3 = basic_tform_cell(rot_euler(0,0,180), s2)
for a in s3:
    a.pos = into_cell(a.pos, s2.cell)
print s3
print s

sg = space_group(s)

eq = []
neq = []
for i in range(len(syms)):
#    s3 = basic_tform_cell(syms[i], s2)
    s3 = deepcopy(s2)
    for a in s3:
        a.pos = into_cell(np.dot(syms[i],a.pos), s2.cell)
#    s3.cell = np.dot(npl.inv(syms[i]),s3.cell)
        
    if (not eq_struct(s,s3)):
        neq.append(i)
    else:
        eq.append(i)
        print syms[i]
print len(eq), "ARE equal"
print len(neq), "are NOT equal"

In [ ]:
ss=  deepcopy(s)
ss.clear()
ss.add_atom(0,0,0,'Au')
print ss
sg = space_group(ss)
print len(sg)
print sg

In [ ]:
sq = pcread.poscar("symtest.square.vasp")
sg = space_group(sq)
print len(sg)
for g in sg:
    print g

In [26]:
s1 = pcread.poscar("symtest.vasp")
s2 = pcread.poscar("symtest.sym.vasp")
#s1 = pcread.poscar("symtest.square.vasp")
#s2 = basic_tform_cell(rot_euler(0,0,90), s1)

s1 = pcread.poscar("badsns_a")
s2 = pcread.poscar("badsns_b")

fcc = Structure([[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
fcc.add_atom(0,0,0,"Au")
grp = space_group(fcc)

def eq_latt(A,B):
    mind = 1e10
    eps = 1e-1
    goodP = []
    perms = [[0,1,2],[0,2,1],[1,0,2],[1,2,0],[2,0,1],[2,1,0]]  ## permute a,b,c in case there are other orders also good
    for i in range(-1,2,2):
        for j in range(-1,2,2):
            for k in range(-1,2,2):
                for p in perms:
                    P = np.zeros((3,3))
                    for m in range(3):
                        P[p[0],0] = i
                        P[p[1],1] = j
                        P[p[2],2] = k
                    diff = npl.norm(np.dot(P,A) - B)
                    if (diff < mind):
                        minP = P
                        mind = diff

                    if diff < eps:
                        print "eq enough latt"
                        print diff
#                        print A
#                        print B
#                        print np.dot(P,A)

                        print P
                        goodP.append(P)
    print mind
    print minP
    return (goodP)
                    
eq = []
neq = []
#for i in range(len(syms)):
#for i in range(10):
for i in range(len(grp)):
    g = grp[i]
    test_cell = np.dot(g[0:3,0:3], s2.cell)
    gp = eq_latt(test_cell, s1.cell)
    if (len(gp) > 0):
        print "symmetry %d has %d perms that look good" % (i, len(gp))
        print g
        print 
        eq.append(i)
    else:
        print "symmetry %d is not a symmetry for this cell" % (i)
        print g
        print 
        neq.append(i)
        
#    for a in s3:
#        a.pos = into_cell(np.dot(syms[i],a.pos), s2.cell)
#    s3.cell = np.dot(npl.inv(syms[i]),s3.cell)
        
print "Found %d syms that could produce matching cells and %d that cannot" % (len(eq),len(neq))

7.47219219373
[[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]
symmetry 0 is not a symmetry for this cell
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]

7.47219219373
[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0. -1.  0.]]
symmetry 1 is not a symmetry for this cell
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]

7.47219219373
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1.  0.  0.]]
symmetry 2 is not a symmetry for this cell
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]

7.47219219373
[[ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0. -1.  0.]]
symmetry 3 is not a symmetry for this cell
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]

7.47219219373
[[ 0.  1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]]
symmetry 4 is not a symmetry for this cell
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]

7.47219219373
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
symmetry 5 is not a symmetry for this cell
[[-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]

7.47219219

In [3]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
for sym in symdict.keys():
    print 
    print sym
    for s in symdict[sym]:
        print s
    

NameError: name 'symdict' is not defined

In [4]:
def eq_sym(s,t):
    eps = 1e-15
    return (npl.norm(s-t) < eps)

def find_sym(s, symset):
    for sym in symset:
        if eq_sym(s, sym):
            return True
    return False

def get_sym_set(symdict):
    symset = []
    for k in symdict.keys():
        for s in symdict[k]:
            if not find_sym(s,symset):
                symset.append(s)
    return symset

In [20]:
symset = get_sym_set(symdict)
for s in symset:
    print s
print len(symset)
print len(symdict)



[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[-1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0. -1.]]
[[ 1. -0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
[[ 1. -0. -0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[ 1.  0. -0.]
 [ 0.  1. -0.]
 [ 0.  0. -1.]]
[[ 0. -1.  0.]
 [ 1. -0.  0.]
 [ 0.  0.  1.]]
[[-0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
[[ 0. -1.  0.]
 [ 1. -0. -0.]
 [ 0.  0. -1.]]
[[-0.  1. -0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
[[ 0.  1. -0.]
 [ 1. -0. -0.]
 [ 0.  0. -1.]]
[[-0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
[[-0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
[[ 0.  1.  0.]
 [ 1. -0.  0.]
 [ 0.  0.  1.]]
[[-0.5    0.866  0.   ]
 [-0.866 -0.5    0.   ]
 [ 0.     0.     1.   ]]
[[-0.5   -0.866  0.   ]
 [ 0.866 -0.5    0.   ]
 [ 0.     0.     1.   ]]
[[ 0.5   -0.866 -0.   ]
 [ 0.866  0.5   -0.   ]
 [ 0.     0.    -1.   ]]
[[ 0.5    0.866 -0.   ]
 [-0.866  0.5   -0.  

In [18]:
from math import acos,pi
acos (.866) * 180./pi

30.002910931188026

In [7]:
print len(syms)
print len(symdict)

192
229


In [36]:
from pyspglib import spglib
s = pcread.poscar("traj0.vasp")
a = pcread.poscar("badsns_a")
b = pcread.poscar("badsns_b")


sg = spglib.get_spacegroup(a, symprec=1e-4, angle_tolerance=2.0)
print sg

sg = spglib.get_spacegroup(b, symprec=1e-4, angle_tolerance=2.0)
print sg

sg = spglib.get_spacegroup(s, symprec=1e-3, angle_tolerance=3.0)
print sg

print a
print b
print s

Ccmm (63)
Pnma (62)
P-1 (2)
Structure( -4.148, 2.55767e-16, 2.074,
           0, 2.55767e-16, 5.74,
           0, 4.177, 0,
           scale=1, name='icsd_100672.cif' )\
  .add_atom(-3.80147e-16, 4.00652, 1.04425, 'Al')\
  .add_atom(-2.074, 1.73348, 3.13275, 'Al')\
  .add_atom(2.85986e-16, 1.3776, 1.04425, 'Sn')\
  .add_atom(-2.074, 4.3624, 3.13275, 'Sn')
Structure( 11.1827, 2.43503e-16, 2.64769e-16,
           0, 3.9767, 2.64769e-16,
           0, 0, 4.324,
           scale=1, name='icsd_651025.cif' )\
  .add_atom(9.5053, 0.994175, 2.15335, 'Al')\
  .add_atom(3.91395, 0.994175, 0.008648, 'Al')\
  .add_atom(1.67741, 2.98252, 2.17065, 'Al')\
  .add_atom(7.26876, 2.98253, 4.31535, 'Al')\
  .add_atom(6.96682, 0.994175, 1.82905, 'Sn')\
  .add_atom(1.37547, 0.994175, 0.332948, 'Sn')\
  .add_atom(4.21588, 2.98252, 2.49495, 'Sn')\
  .add_atom(9.80723, 2.98253, 3.99105, 'Sn')
Structure( 3.9767, -2.64769e-16, 0,
           0, 4.324, 0,
           2.43503e-16, 2.64769e-16, 11.1827,
           sc

In [5]:
s1 = pcread.poscar("symtest.vasp")
s2 = pcread.poscar("symtest.sym.vasp")

print s1
print s2
from pylada.crystal import Structure
s3 = Structure(s2.cell)
s3.add_atom(0,0,0, 'Au')
grp = space_group(s3)
for g in grp:
    print g
    stest = deepcopy(s2)
    for a in stest:
        a.pos = into_cell(np.dot(g[0:3,0:3], a.pos), stest.cell)
    print eq_struct(stest, s1)

Structure( 10, 5, 0,
           5, 10, 0,
           0, 0, 10,
           scale=1, name='icsd_100672.cif' )\
  .add_atom(8.5, 5, 0, 'Al')\
  .add_atom(4.5, 4.5, 0, 'Sn')
Structure( 10, 5, 0,
           5, 10, 0,
           0, 0, 10,
           scale=1, name='icsd_100672.cif' )\
  .add_atom(6.5, 10, 0, 'Al')\
  .add_atom(10.5, 10.5, 0, 'Sn')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
False
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
True
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
True
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
False
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
False
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
False
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
False
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
False


In [14]:
s1 = pcread.poscar("badsns_a")
s2 = pcread.poscar("badsns_b")
print s1.cell
print s2.cell
s3 = Structure(s1.cell)
s3.add_atom(0,0,0, 'Au')
tol = 10.
grp = space_group(s3,tol)
print "sns_A"
print len(grp)
s3 = Structure(s2.cell)
s3.add_atom(0,0,0, 'Au')
grp = space_group(s3, tol)
print "sns_B"
print len(grp)



[[-4.148  0.     2.074]
 [ 0.     0.     5.74 ]
 [ 0.     4.177  0.   ]]
[[ 11.183   0.      0.   ]
 [  0.      3.977   0.   ]
 [  0.      0.      4.324]]
sns_A
8
sns_B
8


In [17]:
fcc = Structure([[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
fcc.add_atom(0,0,0,"Au")
grp = space_group(fcc)
print len(grp)
for g in grp:
    print g

48
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
[[-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]
[[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[ 0. -1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]
[[ 0.  0. -1.]
 [-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[ 0.  0. -1.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]
[[ 0.  0.  1.]
 [-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[ 0.  0.  1.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]
[[-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
[[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
[[ 0. -1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]
 [ 0.  0.  0.]]
[[ 0. -1.  0.]
 [ 0. 

In [29]:
print s1
print s2

Structure( -4.148, 2.55767e-16, 2.074,
           0, 2.55767e-16, 5.74,
           0, 4.177, 0,
           scale=1, name='icsd_100672.cif' )\
  .add_atom(-3.80147e-16, 4.00652, 1.04425, 'Al')\
  .add_atom(-2.074, 1.73348, 3.13275, 'Al')\
  .add_atom(2.85986e-16, 1.3776, 1.04425, 'Sn')\
  .add_atom(-2.074, 4.3624, 3.13275, 'Sn')
Structure( 11.1827, 2.43503e-16, 2.64769e-16,
           0, 3.9767, 2.64769e-16,
           0, 0, 4.324,
           scale=1, name='icsd_651025.cif' )\
  .add_atom(9.5053, 0.994175, 2.15335, 'Al')\
  .add_atom(3.91395, 0.994175, 0.008648, 'Al')\
  .add_atom(1.67741, 2.98252, 2.17065, 'Al')\
  .add_atom(7.26876, 2.98253, 4.31535, 'Al')\
  .add_atom(6.96682, 0.994175, 1.82905, 'Sn')\
  .add_atom(1.37547, 0.994175, 0.332948, 'Sn')\
  .add_atom(4.21588, 2.98252, 2.49495, 'Sn')\
  .add_atom(9.80723, 2.98253, 3.99105, 'Sn')


In [39]:
from pylada.crystal import neighbors
help(neighbors)

Help on built-in function neighbors in module pylada.crystal.cppwrappers:

neighbors(...)
    Returns list of neighbors to input position 
    
    Creates a list referencing neighbors of a given position in a structure.
    In order to make this function well defined, it may return more
    atoms that actually requested. For instance, in an fcc structure
    with center at the origin, if asked for the 6 first neighbors,
    actually the first twelve are returned since they are equidistant.
    The input tolerance is the judge of equidistance.
    
    :param structure:
        :class:`Structure` from which to determine neighbors.
    :param nmax:
        Integer number of first neighbors to search for.
    :param center:
        Position for which to determine first neighbors.
    :param tolerance:
        Tolerance criteria for judging equidistance.
    
    :returns: A list of 3-tuples. The first item is a refence to the
              neighboring atom, the second is the position of 

In [41]:
from pylada.crystal import coordination_shells as coord
help(coord)

Help on built-in function coordination_shells in module pylada.crystal.cppwrappers:

coordination_shells(...)
    Creates list of coordination shells up to given order.
    
    :param structure:
        :class:`Structure` from which to determine neighbors.
    :param nshells:
        Integer number of shells to compute.
    :param center:
        Position for which to determine first neighbors.
    :param tolerance:
        Tolerance criteria for judging equidistance.
    
    :param natoms:
        Integer number of neighbors to consider. Defaults to fcc + some
        security.
    
    :returns: A list of lists of tuples. The outer list is over
              coordination shells.  The inner list references the atoms
              in a shell.  Each innermost tuple contains a reference to
              the atom in question, a vector from the center to the
              relevant periodic image of the atom, and finally, the
              associated distance.

